## Converting object-labeled wisdom dataset to cluttered omniglot format

In [75]:
import os
import cv2
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [76]:
from dataset_utils import mkdir_if_missing
from PIL import Image
from skimage import io
from tqdm import tqdm
import pprint
import json

In [77]:
# input directories
AMODAL_MASK_DIR = "/nfs/diskstation/dmwang/labeled_wisdom_real/dataset"
SCENE_DIR = "/nfs/diskstation/dmwang/labeled_wisdom_real/phoxi/depth_ims"
JSON_DIR = "/nfs/diskstation/dmwang/labeled_wisdom_real/phoxi/color_ims"
MASK_DIR = "/nfs/diskstation/dmwang/labeled_wisdom_real/phoxi/modal_segmasks"

# output directories
OUT_DIR = "/nfs/diskstation/projects/dex-net/segmentation/datasets/mask-net-real/fold_0000"
mkdir_if_missing(OUT_DIR)
mkdir_if_missing(os.path.join(OUT_DIR, "train"))
mkdir_if_missing(os.path.join(OUT_DIR, "val-train"))
mkdir_if_missing(os.path.join(OUT_DIR, "val-one-shot"))
mkdir_if_missing(os.path.join(OUT_DIR, "test-train"))
mkdir_if_missing(os.path.join(OUT_DIR, "test-one-shot"))

# real dataset parameters
NUM_IMS = 400

# original image shape
IM_WIDTH = 772
IM_HEIGHT = 1032


# 1:3 ratio between im_size and tar_size
IM_SIZE = 384
TAR_SIZE = 128

# Image distortion
ANGLE = 100
SHEAR = 4

# For storage purposes
BLOCK_SIZE = 500

In [78]:
def rot_x(phi, theta, ptx, pty):
    return np.cos(phi+theta)*ptx + np.sin(phi-theta)*pty


def rot_y(phi, theta, ptx, pty):
    return -np.sin(phi+theta)*ptx + np.cos(phi-theta)*pty


def prepare_img(img, angle=100, shear=2.5, scale=2):
    # Apply affine transformations and scale characters for data augmentation
    phi = np.radians(np.random.uniform(-angle, angle))
    theta = np.radians(np.random.uniform(-shear, shear))
    a = scale**np.random.uniform(-1, 1)
    b = scale**np.random.uniform(-1, 1)
    (x, y) = img.shape
    x = a * x
    y = b * y
    xextremes = [rot_x(phi, theta, 0, 0), rot_x(phi, theta, 0, y), rot_x(phi, theta, x, 0), rot_x(phi, theta, x, y)]
    yextremes = [rot_y(phi, theta, 0, 0), rot_y(phi, theta, 0, y), rot_y(phi, theta, x, 0), rot_y(phi, theta, x, y)]
    mnx = min(xextremes)
    mxx = max(xextremes)
    mny = min(yextremes)
    mxy = max(yextremes)

    aff_bas = np.array([[a*np.cos(phi+theta), b*np.sin(phi-theta), -mnx], [-a*np.sin(phi+theta), b*np.cos(phi-theta), -mny], [0, 0, 1]])
    aff_prm = np.linalg.inv(aff_bas)
    pil_img = Image.fromarray(img)
    pil_img = pil_img.transform((int(mxx - mnx),int(mxy - mny)),
                                    method=Image.AFFINE,
                                    data=np.ndarray.flatten(aff_prm[0:2, :]))
    pil_img = pil_img.resize((int(TAR_SIZE * (mxx - mnx) / 100), int(TAR_SIZE * (mxy - mny) / 100)))

    return np.array(pil_img)


def bbox(im):
    # get bounding box coordinates
    rows = np.any(im, axis=1)
    cols = np.any(im, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]

    return rmin, rmax, cmin, cmax


def make_target(modal_mask, angle=0, shear=0, scale=1):
    # make target image
    transformed_mask = prepare_img(modal_mask, angle, shear, scale)
    top, bot, left, right = bbox(transformed_mask)
    obj_size = max(bot - top, right - left)
    margin = max((TAR_SIZE * 2 - obj_size) // 2, 0)
    return cv2.resize(
        transformed_mask[max(0, top - margin):min(transformed_mask.shape[0], bot + margin),
                         max(0, left - margin):min(transformed_mask.shape[1], right + margin)],
        (TAR_SIZE, TAR_SIZE),
        interpolation=cv2.INTER_NEAREST)

def resize_scene(im):
    if len(im.shape) == 2:
        im = np.pad(im, (((IM_WIDTH - IM_SIZE) // 2, (IM_WIDTH - IM_SIZE) // 2), (0, 0)), mode="constant")
    elif len(im.shape) == 3:
        im = np.pad(im, (((IM_WIDTH - IM_SIZE) // 2, (IM_WIDTH - IM_SIZE) // 2), (0, 0), (0, 0)), mode="constant")
    else:
        raise Exception("image dimensions not valid for scene/ground truth, shape: {}".format(im.shape))
    return cv2.resize(
        im,
        (IM_SIZE, IM_SIZE),
        interpolation=cv2.INTER_NEAREST)

In [80]:
# Looping through all image indices
#   Looping through all labels in the json list
#     Get the name
#     Get the target file corresponding to the name
#     Add the image to the batch
#     Process the segmask from modal_segmasks

data_count = 0
for meta_idx in tqdm(range(NUM_IMS * 30)):
    idx = meta_idx % NUM_IMS
    f = open(os.path.join(JSON_DIR, "image_{:06d}.json".format(idx)))
    json_file = json.load(f)
    scene_path = os.path.join(SCENE_DIR, "image_{:06d}.png".format(idx))
    mask_path = os.path.join(MASK_DIR, "image_{:06d}.png".format(idx))

    # read and blockify scene
    scene_im = io.imread(scene_path)
    scene_im = resize_scene(scene_im)
    scene_im = np.expand_dims(scene_im, axis=0)
    
    # for modal masks
    joint_mask = io.imread(mask_path)
    for label in json_file["labels"]:
        target_name = label["label_class"]
        target_id = label["object_id"]
        target_path = os.path.join(AMODAL_MASK_DIR, 
                                   "image_{:06d}".format(idx), 
                                   "{}.png".format(target_name))
        try:
            target_im = io.imread(target_path)
            amodal_mask = make_target(target_im[...,0], angle=0, shear=0)
        except:
            continue
        
        modal_mask = np.copy(joint_mask)
        modal_mask[modal_mask != target_id] = 0
        modal_mask = resize_scene(modal_mask)
        modal_mask[modal_mask > 0] = 1
        data_count += 1

        """plt.imshow(scene_im[0])
        plt.show()
        plt.imshow(modal_mask)
        plt.show()
        plt.imshow(amodal_mask)
        plt.show()"""
        
        # triplicate the amodal mask (128, 128) --> (128, 128, 3)
        amodal_mask = np.stack([amodal_mask, amodal_mask, amodal_mask], axis=2)
 
        
        # create 1-sized blocks
        amodal_mask = np.expand_dims(amodal_mask, axis=0)
        modal_mask = np.expand_dims(modal_mask, axis=2)
        modal_mask = np.expand_dims(modal_mask, axis=0)
        
        """print(modal_mask.shape)
        print(amodal_mask.shape)
        print(scene_im.shape)"""
        
        np.save(os.path.join(
            OUT_DIR,
            "test-train/",
            "image_{:08d}.npy".format(meta_idx)),
               scene_im)
        np.save(os.path.join(
            OUT_DIR,
            "test-train/",
            "segmentation_{:08d}.npy".format(meta_idx)),
               modal_mask)
        np.save(os.path.join(
            OUT_DIR,
            "test-train/",
            "target_{:08d}.npy".format(meta_idx)),
               amodal_mask)
        np.save(os.path.join(
            OUT_DIR,
            "test-one-shot/",
            "image_{:08d}.npy".format(meta_idx)),
               scene_im)
        np.save(os.path.join(
            OUT_DIR,
            "test-one-shot/",
            "segmentation_{:08d}.npy".format(meta_idx)),
               modal_mask)
        np.save(os.path.join(
            OUT_DIR,
            "test-one-shot/",
            "target_{:08d}.npy".format(meta_idx)),
               amodal_mask)        
        
            
        
        










  0%|          | 0/12000 [00:00<?, ?it/s]







  0%|          | 1/12000 [00:00<40:42,  4.91it/s]







  0%|          | 2/12000 [00:00<35:58,  5.56it/s]







  0%|          | 3/12000 [00:00<34:45,  5.75it/s]







  0%|          | 4/12000 [00:00<35:42,  5.60it/s]







  0%|          | 5/12000 [00:00<34:02,  5.87it/s]







  0%|          | 6/12000 [00:00<30:55,  6.46it/s]







  0%|          | 7/12000 [00:01<35:02,  5.71it/s]







  0%|          | 8/12000 [00:01<35:30,  5.63it/s]







  0%|          | 9/12000 [00:01<36:13,  5.52it/s]







  0%|          | 10/12000 [00:01<34:31,  5.79it/s]







  0%|          | 11/12000 [00:01<33:56,  5.89it/s]







  0%|          | 12/12000 [00:02<37:06,  5.38it/s]







  0%|          | 13/12000 [00:02<37:11,  5.37it/s]







  0%|          | 14/12000 [00:02<39:23,  5.07it/s]







  0%|          | 16/12000 [00:02<35:39,  5.60it/s]







  0%|          | 17/12000 [00:02<34:04,  5.86it/s]







  0%|          | 18/1200

  1%|          | 110/12000 [00:20<33:52,  5.85it/s]







  1%|          | 111/12000 [00:20<40:48,  4.86it/s]







  1%|          | 112/12000 [00:21<46:05,  4.30it/s]







  1%|          | 113/12000 [00:21<44:27,  4.46it/s]







  1%|          | 114/12000 [00:21<48:20,  4.10it/s]







  1%|          | 115/12000 [00:21<45:25,  4.36it/s]







  1%|          | 116/12000 [00:21<39:12,  5.05it/s]







  1%|          | 117/12000 [00:22<37:33,  5.27it/s]







  1%|          | 118/12000 [00:22<33:35,  5.90it/s]







  1%|          | 119/12000 [00:22<31:52,  6.21it/s]







  1%|          | 120/12000 [00:22<35:33,  5.57it/s]







  1%|          | 121/12000 [00:22<32:40,  6.06it/s]







  1%|          | 122/12000 [00:22<33:19,  5.94it/s]







  1%|          | 123/12000 [00:23<33:31,  5.90it/s]







  1%|          | 124/12000 [00:23<30:47,  6.43it/s]







  1%|          | 125/12000 [00:23<37:35,  5.26it/s]







  1%|          | 126/12000 [00:23<35:53,  5.51it/s]






  2%|▏         | 214/12000 [00:41<45:50,  4.29it/s]







  2%|▏         | 215/12000 [00:41<41:06,  4.78it/s]







  2%|▏         | 216/12000 [00:41<37:44,  5.20it/s]







  2%|▏         | 217/12000 [00:41<34:27,  5.70it/s]







  2%|▏         | 218/12000 [00:41<33:52,  5.80it/s]







  2%|▏         | 220/12000 [00:42<30:51,  6.36it/s]







  2%|▏         | 221/12000 [00:42<30:52,  6.36it/s]







  2%|▏         | 222/12000 [00:42<37:19,  5.26it/s]







  2%|▏         | 223/12000 [00:42<39:38,  4.95it/s]







  2%|▏         | 225/12000 [00:43<37:15,  5.27it/s]







  2%|▏         | 226/12000 [00:43<38:44,  5.07it/s]







  2%|▏         | 228/12000 [00:43<33:20,  5.89it/s]







  2%|▏         | 230/12000 [00:43<32:03,  6.12it/s]







  2%|▏         | 231/12000 [00:44<34:05,  5.75it/s]







  2%|▏         | 232/12000 [00:44<31:18,  6.27it/s]







  2%|▏         | 233/12000 [00:44<33:07,  5.92it/s]







  2%|▏         | 234/12000 [00:44<30:35,  6.41it/s]






  3%|▎         | 325/12000 [01:00<39:07,  4.97it/s]







  3%|▎         | 326/12000 [01:00<36:58,  5.26it/s]







  3%|▎         | 327/12000 [01:01<41:06,  4.73it/s]







  3%|▎         | 328/12000 [01:01<40:08,  4.85it/s]







  3%|▎         | 330/12000 [01:01<36:28,  5.33it/s]







  3%|▎         | 331/12000 [01:01<40:50,  4.76it/s]







  3%|▎         | 332/12000 [01:02<43:58,  4.42it/s]







  3%|▎         | 333/12000 [01:02<44:29,  4.37it/s]







  3%|▎         | 334/12000 [01:02<44:39,  4.35it/s]







  3%|▎         | 335/12000 [01:02<40:34,  4.79it/s]







  3%|▎         | 336/12000 [01:03<37:42,  5.15it/s]







  3%|▎         | 337/12000 [01:03<37:48,  5.14it/s]







  3%|▎         | 338/12000 [01:03<37:57,  5.12it/s]







  3%|▎         | 339/12000 [01:03<38:09,  5.09it/s]







  3%|▎         | 340/12000 [01:03<35:56,  5.41it/s]







  3%|▎         | 341/12000 [01:04<38:26,  5.06it/s]







  3%|▎         | 342/12000 [01:04<36:34,  5.31it/s]






  4%|▎         | 437/12000 [01:21<37:46,  5.10it/s]







  4%|▎         | 438/12000 [01:21<41:49,  4.61it/s]







  4%|▎         | 439/12000 [01:21<38:50,  4.96it/s]







  4%|▎         | 440/12000 [01:22<42:12,  4.56it/s]







  4%|▎         | 441/12000 [01:22<36:55,  5.22it/s]







  4%|▎         | 443/12000 [01:22<32:04,  6.01it/s]







  4%|▎         | 444/12000 [01:22<38:11,  5.04it/s]







  4%|▎         | 445/12000 [01:22<35:47,  5.38it/s]







  4%|▎         | 446/12000 [01:23<36:45,  5.24it/s]







  4%|▎         | 447/12000 [01:23<39:07,  4.92it/s]







  4%|▎         | 448/12000 [01:23<34:45,  5.54it/s]







  4%|▎         | 449/12000 [01:23<35:52,  5.37it/s]







  4%|▍         | 451/12000 [01:24<35:17,  5.45it/s]







  4%|▍         | 452/12000 [01:24<33:40,  5.71it/s]







  4%|▍         | 453/12000 [01:24<34:35,  5.56it/s]







  4%|▍         | 454/12000 [01:24<31:29,  6.11it/s]







  4%|▍         | 455/12000 [01:24<31:13,  6.16it/s]






...

 96%|█████████▌| 11517/12000 [38:26<01:32,  5.22it/s]







 96%|█████████▌| 11518/12000 [38:26<01:22,  5.82it/s]







 96%|█████████▌| 11519/12000 [38:28<04:41,  1.71it/s]







 96%|█████████▌| 11520/12000 [38:28<03:34,  2.24it/s]







 96%|█████████▌| 11521/12000 [38:28<02:50,  2.80it/s]







 96%|█████████▌| 11522/12000 [38:28<02:22,  3.34it/s]







 96%|█████████▌| 11523/12000 [38:28<01:58,  4.03it/s]







 96%|█████████▌| 11524/12000 [38:28<01:53,  4.20it/s]







 96%|█████████▌| 11525/12000 [38:29<01:57,  4.03it/s]







 96%|█████████▌| 11526/12000 [38:29<01:44,  4.52it/s]







 96%|█████████▌| 11527/12000 [38:29<01:52,  4.19it/s]







 96%|█████████▌| 11528/12000 [38:29<01:45,  4.49it/s]







 96%|█████████▌| 11530/12000 [38:30<01:33,  5.03it/s]







 96%|█████████▌| 11531/12000 [38:30<01:41,  4.62it/s]







 96%|█████████▌| 11532/12000 [38:30<01:51,  4.19it/s]







 96%|█████████▌| 11533/12000 [38:30<01:50,  4.21it/s]







 96%|█████████▌| 11534/1

 97%|█████████▋| 11625/12000 [38:49<00:52,  7.14it/s]







 97%|█████████▋| 11626/12000 [38:49<01:06,  5.59it/s]







 97%|█████████▋| 11627/12000 [38:49<01:04,  5.76it/s]







 97%|█████████▋| 11628/12000 [38:49<00:58,  6.31it/s]







 97%|█████████▋| 11629/12000 [38:49<00:58,  6.34it/s]







 97%|█████████▋| 11631/12000 [38:50<00:57,  6.42it/s]







 97%|█████████▋| 11632/12000 [38:50<00:53,  6.87it/s]







 97%|█████████▋| 11633/12000 [38:50<01:09,  5.29it/s]







 97%|█████████▋| 11634/12000 [38:50<01:10,  5.20it/s]







 97%|█████████▋| 11635/12000 [38:50<01:05,  5.57it/s]







 97%|█████████▋| 11636/12000 [38:51<01:18,  4.66it/s]







 97%|█████████▋| 11637/12000 [38:51<01:10,  5.14it/s]







 97%|█████████▋| 11638/12000 [38:51<01:17,  4.67it/s]







 97%|█████████▋| 11639/12000 [38:51<01:11,  5.04it/s]







 97%|█████████▋| 11640/12000 [38:52<01:16,  4.68it/s]







 97%|█████████▋| 11641/12000 [38:52<01:06,  5.40it/s]







 97%|█████████▋| 11643/1

 98%|█████████▊| 11728/12000 [39:08<00:41,  6.61it/s]







 98%|█████████▊| 11729/12000 [39:09<00:39,  6.91it/s]







 98%|█████████▊| 11730/12000 [39:09<00:45,  5.89it/s]







 98%|█████████▊| 11731/12000 [39:09<00:42,  6.39it/s]







 98%|█████████▊| 11732/12000 [39:09<00:56,  4.76it/s]







 98%|█████████▊| 11733/12000 [39:09<00:52,  5.09it/s]







 98%|█████████▊| 11735/12000 [39:10<00:43,  6.13it/s]







 98%|█████████▊| 11736/12000 [39:10<00:40,  6.58it/s]







 98%|█████████▊| 11737/12000 [39:10<00:40,  6.46it/s]







 98%|█████████▊| 11738/12000 [39:10<00:37,  6.90it/s]







 98%|█████████▊| 11740/12000 [39:10<00:38,  6.72it/s]







 98%|█████████▊| 11741/12000 [39:10<00:36,  7.05it/s]







 98%|█████████▊| 11742/12000 [39:11<00:43,  5.96it/s]







 98%|█████████▊| 11743/12000 [39:11<00:50,  5.11it/s]







 98%|█████████▊| 11744/12000 [39:11<00:50,  5.10it/s]







 98%|█████████▊| 11745/12000 [39:11<00:47,  5.39it/s]







 98%|█████████▊| 11746/1

 99%|█████████▊| 11834/12000 [39:30<00:26,  6.16it/s]







 99%|█████████▊| 11836/12000 [39:30<00:26,  6.14it/s]







 99%|█████████▊| 11837/12000 [39:30<00:33,  4.89it/s]







 99%|█████████▊| 11838/12000 [39:31<00:31,  5.19it/s]







 99%|█████████▊| 11839/12000 [39:31<00:30,  5.34it/s]







 99%|█████████▊| 11840/12000 [39:31<00:27,  5.92it/s]







 99%|█████████▊| 11841/12000 [39:31<00:31,  5.08it/s]







 99%|█████████▊| 11842/12000 [39:31<00:27,  5.72it/s]







 99%|█████████▊| 11843/12000 [39:32<00:29,  5.39it/s]







 99%|█████████▊| 11844/12000 [39:32<00:28,  5.55it/s]







 99%|█████████▊| 11845/12000 [39:32<00:25,  6.10it/s]







 99%|█████████▊| 11846/12000 [39:32<00:30,  5.11it/s]







 99%|█████████▊| 11848/12000 [39:32<00:28,  5.30it/s]







 99%|█████████▊| 11849/12000 [39:33<00:33,  4.53it/s]







 99%|█████████▉| 11850/12000 [39:33<00:30,  4.92it/s]







 99%|█████████▉| 11851/12000 [39:33<00:33,  4.51it/s]







 99%|█████████▉| 11852/1

100%|█████████▉| 11940/12000 [39:50<00:11,  5.13it/s]







100%|█████████▉| 11941/12000 [39:50<00:13,  4.43it/s]







100%|█████████▉| 11942/12000 [39:50<00:12,  4.70it/s]







100%|█████████▉| 11943/12000 [39:50<00:10,  5.19it/s]







100%|█████████▉| 11944/12000 [39:50<00:11,  5.08it/s]







100%|█████████▉| 11945/12000 [39:51<00:11,  4.94it/s]







100%|█████████▉| 11946/12000 [39:51<00:09,  5.60it/s]







100%|█████████▉| 11947/12000 [39:51<00:09,  5.60it/s]







100%|█████████▉| 11948/12000 [39:51<00:10,  5.01it/s]







100%|█████████▉| 11949/12000 [39:51<00:10,  5.02it/s]







100%|█████████▉| 11950/12000 [39:52<00:09,  5.00it/s]







100%|█████████▉| 11951/12000 [39:52<00:10,  4.72it/s]







100%|█████████▉| 11952/12000 [39:52<00:10,  4.52it/s]







100%|█████████▉| 11953/12000 [39:52<00:09,  5.16it/s]







100%|█████████▉| 11954/12000 [39:52<00:09,  4.73it/s]







100%|█████████▉| 11955/12000 [39:53<00:08,  5.58it/s]







100%|█████████▉| 11956/1

In [16]:
im.shape

NameError: name 'im' is not defined

In [ ]:
np.unique(im)

In [ ]:
im2 = np.copy(im)
im2[im2 != 1] = 0
print(im2.shape)

In [ ]:
plt.imshow(im2)
